In [1]:
#pip install opencv-python

In [2]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from textblob import TextBlob
import lyricsgenius

In [3]:

client_credentials_manager = SpotifyClientCredentials(client_id='INCLUDE_CLIENT_ID', client_secret='INCLUDE_SECRET_ID')
scope = "user-read-recently-played"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id='INCLUDE_CLIENT_ID', client_secret='INCLUDE_SECRET_ID', redirect_uri='https://open.spotify.com/'))

#this is used to get lyrics to perform analysis
genius = lyricsgenius.Genius('cJJDWpZwnLSPHD3Yrp6nKSJAS8Ct6da1NDLQEvyZoz1NAIeABj9fmjgi8COuQI2d')


In [23]:
import cv2
import numpy as np
from keras.models import model_from_json


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('emotion_detection_model_100epochs.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

emotion_model.load_weights("emotion_detection_model_100epochs.h5")
print("Loaded model from disk")

# start the webcam feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        emotion=emotion_dict[maxindex]
    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print (emotion)
cap.release()
cv2.destroyAllWindows()

Loaded model from disk
1/1 [==============================] - 2s 2s/step
Neutral


In [16]:
print(emotion)
if emotion == "Happy":
    recommended_track = sp.search(q='genre:"pop"', type='track')['tracks']['items'][0]
elif emotion == "Sad":
    recommended_track = sp.search(q='genre:"soul"', type='track')['tracks']['items'][0]
elif emotion == "Angry":
    recommended_track = sp.search(q='genre:"metal"', type='track')['tracks']['items'][0]
elif emotion == "Disgusted":
    recommended_track = sp.search(q='genre:"alternative"', type='track')['tracks']['items'][0]
elif emotion == "Neutral":
    sentiment_polarity=perform_sentiment_analysis
elif emotion == "Surprised":
    recommended_track = sp.search(q='genre:"trending"', type='track')['tracks']['items'][0]
elif emotion == "Fearful":
    recommended_track = sp.search(q='genre:"punk"', type='track')['tracks']['items'][0]

Neutral


In [18]:
if emotion != "Neutral":
    print(f"Based on your recent emotions, we recommend: {recommended_track['name']} by {recommended_track['artists'][0]['name']}")
else:
    results = sp.current_user_recently_played(limit=2)
    recent_tracks = [item['track']['name'] for item in results['items']]
    recent_lyrics = []
    for track in recent_tracks:
        song = genius.search_song(track)
        if song is not None:
            recent_lyrics.append(song.lyrics)
    recent_lyrics_string = ' '.join(recent_lyrics)
    sentiment = TextBlob(recent_lyrics_string).sentiment.polarity
    if sentiment > 0:
        recommended_track = sp.search(q='genre:"happy"', type='track')['tracks']['items'][0]
    else:
        recommended_track = sp.search(q='genre:"sad" ', type='track')['tracks']['items'][0]
    print(f"Based on your recent emotions, we recommend: {recommended_track['name']} by {recommended_track['artists'][0]['name']}")

Searching for "STAY (with Justin Bieber)"...
Done.
Searching for "Love Yourself"...
Done.
Based on your recent emotions, we recommend: Jumping All over the World by Scooter
